In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.graph_objs as go
import plotly.plotly as py
import plotly
import pandas as pd
import numpy as np
from math import sin,radians,pi

app = dash.Dash()
#
# df = pd.read_csv(
#     'https://gist.githubusercontent.com/chriddyp/'
#     'cb5392c35661370d95f300086accea51/raw/'
#     '8e0768211f6b747c0db42a9ce9a0937dafcbd8b2/'
#     'indicators.csv')

#available_indicators = df['Indicator Name'].unique()
#
# #############################################################
# #Declaring all fan design INPUT variables                   #
# ID=18*0.0254        #inlet blade diameter in meters                 #
# OD=21*0.0254*2        #outlet blade diameter in meters                #
# Beta1=45    #inlet blade angle in degrees                   #
# Beta2=15    #outlet blade angle in degrees                  #
# BetaM=0    # angle at arbitrary middle point between ID&OD
# B1=10*0.0254      #inlet blade height in meters                   #
# B2=6*0.0254      #outlet blade height in meters                  #
# Z= 7       #number of blades                               #
# S=0.125*0.0254    #blade thickness in meters
# Qt=6        #desired target flow rate in m3/s              #
# Pt=3000      #desired target Pressure in Pa
# N=30        #rotational speed in rotation per seconds       #
# K=6        #system resistance curve constant DP=k*Q^2      #
# #############################################################
# #Declaring contants
# rho=1.225   #density of air in kg/s
# dqx=1100      # discretisation of flow , number of points
# drx=500         # discretisation radial , number of points
#
# ##############################################
# #defining funtions to be used in script
# def find_nearest(array,value):
#     #find the nearest value in the array and return the index
#     idx = (np.abs(array-value)).argmin()
#     return idx
#
# ###########################################################
# #Static Pressure/flow rate and Power/flow rate graph
#Qmax=B2*N*(pi*OD)**2*sin(radians(Beta2)) # max flow rate at this outlet angle
# def DPQ(OD,ID,Beta2,B2,N,K):
#     Qmax=B2*N*(pi*OD)**2 # max flow rate
#     #print(Qmax)
#     DPmax=0.5*rho*(pi*OD*N)**2 # max pressure
#     Q=np.linspace(0,Qmax,dqx)
#     DP=0.5*rho*((pi*OD*N)**2-(Q/(pi*OD*B2*sin(radians(Beta2))))**2)
#     DP_Beta5=0.5*rho*((pi*OD*N)**2-(Q/(pi*OD*B2*sin(radians(5))))**2)
#     DP_Beta90=0.5*rho*((pi*OD*N)**2-(Q/(pi*OD*B2*sin(radians(90))))**2)
#     DPk=K*Q**2 # system resistance curve
#
#     return {Q,DP,DP_Beta5,DP_Beta90,DPk}

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}
layout = go.Layout(
    xaxis=dict(
        range=[2, 5]
    ),
    yaxis=dict(
        range=[2, 5]
    ))
app.layout =html.Div([
            html.H1(children='Hello Dash',style={'textAlign': 'center','color': colors['text']}  ),
            html.Label('OD slider'),
            dcc.Slider(id='ODs',min=0,max=10.0,step=1,value=1),
            html.Label('Beta2 slider'),
            dcc.Slider(id='Beta2s',min=0,max=180,value=30,marks={str(i): str(i) for i in range(0,180,30) }),
            html.Label('OD'),
            dcc.Input(id='OD',type='number',inputmode='numeric',value=45.0*0.0254),
            html.Label('ID'),
            dcc.Input(id='ID',type='number',inputmode='numeric',value=25.0*0.0254),
            html.Label('B2'),
            dcc.Input(id='B2',type='number',inputmode='numeric',value=5.0*0.0254),
            html.Label('N'),
            dcc.Input(id='N',type='number',inputmode='numeric',value=30.0),
            html.Label('K'),
            dcc.Input(id='K',type='number',inputmode='numeric',value=2.0),

            dcc.Graph(id='indicator-graphic'),
#                 figure={
#                 'layout': go.Layout(
#                     xaxis={'type': 'log', 'title': 'GDP Per Capita'},
#                     yaxis={'title': 'Life Expectancy', 'range': [20, 90]},
#                     margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
# #                    legend={'x': 0, 'y': 1},
#                     hovermode='closest'
#                 )
#                 }
#             ),
            dcc.Graph(id='indicator-graphic2'),
            dcc.Graph(id='indicator-graphic3')

            ],
            style={'width': '48%', 'display': 'inline-block'}
            )

@app.callback(
    dash.dependencies.Output('indicator-graphic', 'figure'),
    [dash.dependencies.Input('OD', 'value'),
     dash.dependencies.Input('ID', 'value'),
     dash.dependencies.Input('Beta2s', 'value'),
     dash.dependencies.Input('B2', 'value'),
     dash.dependencies.Input('N', 'value'),
     dash.dependencies.Input('K', 'value')]
)
def DPQ(OD,ID,Beta2s,B2,N,K):
    rho=1.225
    dqx=100
    OD=float(OD)
    ID=float(ID)
    B2=float(B2)
    N=float(N)
    K=float(K)

    Qmax=B2*N*(np.pi*OD)**2 # max flow rate
    #print(Qmax)
    DPmax=0.5*rho*(np.pi*OD*N)**2 # max pressure
    Q=np.linspace(0,Qmax,dqx)
    DP=0.5*rho*((np.pi*OD*N)**2-(Q/(np.pi*OD*B2*np.sin(np.radians(Beta2s))))**2)
    DP_Beta5=0.5*rho*((np.pi*OD*N)**2-(Q/(np.pi*OD*B2*np.sin(np.radians(5))))**2)
    DP_Beta90=0.5*rho*((np.pi*OD*N)**2-(Q/(np.pi*OD*B2*np.sin(np.radians(90))))**2)
    DPk=K*Q**2 # system resistance curve

    return {
        'data': [
        go.Scatter(x=Q,y=DP,mode='lines',name='Current'),
        go.Scatter(x=Q,y=DP_Beta5,mode='lines',name='Beta 5'),
        go.Scatter(x=Q,y=DP_Beta90,mode='lines',name='Beta 90'),
        go.Scatter(x=Q,y=DPk,mode='lines',name='System resistance')
        ],

        'layout': go.Layout(
            xaxis={ 'title': 'Flow m3/s'},
            yaxis={'title': 'Pressure Pa', 'range': [0,DP[0]+10]},
            #margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
            legend={'x': 1, 'y': 1},
            hovermode='closest'
        )
    }

# def update_graph(ODs,Beta2s):
#     Q=np.linspace(0,100,100)
#     DP=Q+ODs+Beta2s
#
#     return {
#         'data': [
#         go.Scatter(
#             x=Q,
#             y=DP,
#             mode='lines'
#         ),
#         go.Scatter(
#             x=Q,
#             y=DP,
#             mode='lines'
#         )
#         ]
#     }
    #plotly.offline.plot(data)


@app.callback(
    dash.dependencies.Output('indicator-graphic3', 'figure'),
    [dash.dependencies.Input('ODs', 'value'),
     dash.dependencies.Input('Beta2s', 'value')]
            )
def update_graph3(ODs,Beta2s):
    Q=np.linspace(0,2*3.14,100)
    DP=np.sin(Q*ODs)

    return {
        'data': [go.Scatter(
            x=Q,
            y=DP,
            mode='lines'
        )]
    }


@app.callback(
    dash.dependencies.Output('indicator-graphic2', 'figure'),
    [dash.dependencies.Input('ID', 'value'),
     dash.dependencies.Input('Beta2s', 'value')]
            )
def update_graph2(B1,Beta2s):
    Q=np.linspace(0,2*3.14,100)
    DP=np.sin(Q*float(B1))

    return {
        'data': [go.Scatter(
            x=Q,
            y=DP,
            mode='lines'
        )]
    }

if __name__ == '__main__':
    app.run_server()


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [20/Nov/2017 20:32:05] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:07] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [20/Nov/2017 20:32:09] "POST /_dash-update-component HTTP/1.1" 2